In [1]:
import pandas as pd
import numpy as np
import random
import os

from functools import reduce


# Questions: 
- how to add the dataset to gitignore?

# SHARK ATTACKS, a project by Roberto Henríquez

# PART I: Data cleaning and exploration

## Step 0 - Basic knowledge
To begin the development of this project, it would be good to hold a minimum understanding of `Shark Attacks`.

As I did not know much about this topic at the day the project started, I have recurred to the shark-attack wiki: https://en.wikipedia.org/wiki/Shark_attack

With this information in mind, below is the process of data exploration, cleaning, and wrangling.


## Step 1 - Defining the dataset path, and importing it to begin basic dataset exploration

In [3]:
# To follow along and access the DataSet, download it from KAGGLE using this link
# https://www.kaggle.com/teajay/global-shark-attacks

# Once you have downloaded the DataSet, change the following `dataset` variable to match the 
# path where you have saved the 'attacks.csv' file.

dataset = 'attacks.csv' 
df = pd.read_csv(dataset, encoding='latin-1')

Now, we will check some basic information about the dataset, in order to formulate a more educated hypothesis which we could actually put to test with the data available.

In [4]:
display(df.shape)# To know the shape of the DF
print(df.drop_duplicates().shape) # Shape when eliminating duplicates

(25723, 24)

(6312, 24)


Here, I notice that the shape of the `df` with no duplicates is very small when compared to the whole `df`.

In [5]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
# I want to take a look at the time structures
df.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
25718            NaN
25719            NaN
25720            NaN
25721            NaN
25722            NaN
Name: Date, Length: 25723, dtype: object

In [ ]:
df_nodupes = df.drop_duplicates()
df_nodupes.Date

In [ ]:
# Now, with a df smaller in size, I want to see what info is there on the last couple
# of columns which have unexplicit
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']]

In [ ]:
# Too many null values... let's count them and let
print(df_nodupes.shape)
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].isnull().sum()

In [ ]:
# If only 1 value in the 'Unnamed: 22' column, and 2 values in the
# 'Unnamed: 22' column, I'll not consider this data for my analysis.
df_nodupes = df_nodupes.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

In [ ]:
# Now we'll look at the columns again
df_nodupes.columns

In [ ]:
# The following columns seemed a little bit rare, so i do a value count to find out what they are about
df_nodupes['Case Number.1'].value_counts()

In [ ]:
df_nodupes['Case Number.2'].value_counts()

In [ ]:
df_nodupes['original order'].value_counts()

In [ ]:
#Since 'original order seems like arbitrary indexes, i'll drop it
df_nodupes = df_nodupes.drop(columns='original order')
df_nodupes.columns

In [ ]:
# I want to check out some of the pdf and href
print(df_nodupes[['pdf', 'href']].isnull().sum())
df_nodupes[['pdf', 'href']]

In [ ]:
# I want to check what is on those pdfs
# A random sample of the column 

# With a FOR loop
for i in range(10):
    e = random.choice(range(1000))
    print(f"index: {e}, link: {df_nodupes.iloc[e]['href']}")

# With Random sample
display(random.sample(list(df_nodupes['href']), 10))

# Below are a couple of the links, when I open them, I have found that they are seem quite structured
# It could be possible to parse them later down the road and use a REGEX to find more data

# I also have ran this column a few times to notice that all pdfs have actually been uploaded to
# the same website and have the same naming structure

# After some random sampling, I've noticed that the following indexes have some mistakes.. 
# a REGEX can be used to fix problems like these 


In [ ]:
print(df_nodupes.iloc[332]['href'])
print(df_nodupes.iloc[324]['href'])
print(df_nodupes.iloc[588]['href'])
print(df_nodupes.iloc[569]['href'])

## It looks still like some of these pdfs are duplicates, even after dropping duplicates :

In [ ]:
# how many times each pdf on the dataframe
df_pdf = df_nodupes["pdf"]
df_pdf.value_counts()

In [ ]:
# drop dupes and compare lengths
df_pdf_nodupes = df_pdf.drop_duplicates()

len(df_pdf) - len(df_pdf_nodupes), 'duped values'

## Since the lengths are not the same, I will check if those duplicated entries are only in this column

In [ ]:
# there are 20 duplicated values on the pdf columns
df_nodupes.duplicated('pdf').value_counts()

In [ ]:
# But only 18 dupes if we take Location into count
df_nodupes.duplicated(['pdf','Location']).value_counts()

### I'll look at the rest of the data now.

In [ ]:
print(df_nodupes.shape)
df_nodupes.duplicated().sum()

In [ ]:
df_nodupes = df_nodupes.drop_duplicates()
df_nodupes.duplicated().sum()

In [ ]:
print(df_nodupes.shape)
df_nodupes[["Date", "Location", "pdf"]]

In [ ]:
df_nodupes.Country.value_counts()

In [ ]:
# While checking the columns 'Species ' and 'Sex ' have unnecesary spaces at the end of the string
# to remove these, and also take out the '(Y/N)' from the column 'Fatal'

In [ ]:
df_label = df_nodupes
df_label.columns = ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2']
df_label.columns

In [ ]:
df_label['Fatal'].value_counts()

In [ ]:
remove_spaces = lambda x:  x.remove(' ') if ' ' in x else x
"""

df_label['Fatal'] = list(
                            map(remove_spaces(
                            df_label['Fatal']),
                            ))
"""
    
df_label['Fatal'].value_counts()


# I want to see the indexes which have a duplicated pdf row
"""

dupes = []
for a,b in list(df_label['pdf'].duplicated().items()):
    if b:
        dupes.append(a)
dupes """

In [ ]:
"""
df_label.loc[dupes]
"""


In [ ]:
# dfx = df_nodupes["pdf"].value_counts() if 

# Transform this to sort the shark species


In [ ]:
list(df_label['Species'].value_counts().items())

In [ ]:
# @@ Use this to fill null values: 
# df_clean["drive"] = df_clean.drive.fillna("NoTransmision")

# Injuries and types of attack
The GSAF categorizes scavenging bites on humans as "questionable incidents."

## PROVOKED
Provoked attacks occur when a human touches, hooks, nets, or otherwise aggravates the animal. Incidents that occur outside of a shark's natural habitat, such as aquariums and research holding-pens, are considered provoked, as are all incidents involving captured sharks. Sometimes humans inadvertently provoke an attack, such as when a surfer accidentally hits a shark with a surf board.

## UNPROVOKED
- Hit-and-run attack
- Sneak Attack
- Bump-and-bite attack 

For more information on how to differentiate PROVOKED vs UNPROVOKED attacks :
https://en.wikipedia.org/wiki/Shark_attack#Types_of_attacks

In [ ]:
# Since there is no column that states if the attack was provoked or not,
# I want to analyze the injury column to distinguish between the cases that were provoked
# and those that were unprovoked.

random.sample(list(df_label.Injury.value_counts().items()),20)

In [ ]:
# Categorizing  Provoked and  Unprovoked attacks
#df_clean.loc[df_clean["trany"].str.startswith("M"),"trany"] = "Manual"

provoked = ['PROVOKED', 'hook', 'shot']
#map(lambda words, x : words in x, provoked, df_nodupes.loc[df_nodupes['Injury'].str])
df_nodupes.loc[df_nodupes['Injury'].str]

In [ ]:
df_nodupes.loc[df_nodupes['Injury'].str]

# df_provoked = np.where(df_nodupes.Injury.isin(provoked), True, False) 

# Passing that categorization to a new PROVOKED COLUMN
df_nodupes['Provoked'] = df_provoked
df_nodupes['Provoked'] 

In [ ]:
display(df.columns) # To know which are the columns in the DF
display(df.count()) # To know how much data are we missin on each column
display(df.dtypes)

In [ ]:
df_label[['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'href',
       'Case Number.1', 'Case Number.2']].head(50)